<a href="https://colab.research.google.com/github/nourdouha/codealpha_tasks/blob/main/task%203%20_tifinagh_hand_written_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

In [ ]:
import os
import numpy as np
import cv2 as cv
import random
import pickle
import time
from datetime import timedelta
import math
# CNN
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras import backend as K
# graph
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
# matrice de conf
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix


In [ ]:
CATEGORIES = ["ya", "yab", "yach", "yad", "yadd", "yae", "yaf", "yag", "yagh",
              "yagw", "yah", "yahh", "yaj", "yak", "yakw", "yal", "yam", "yan",
              "yaq", "yar","yarr", "yas", "yass", "yat", "yatt", "yaw", "yax",
              "yay", "yaz", "yazz", "yey", "yi", "yu"] # a list of all possible classes
T_CATEGORIES = ['ⴰ', 'ⴱ', 'ⵛ', 'ⴷ', 'ⴹ', 'ⵄ', 'ⴼ', 'ⴳ', 'ⵖ', 'ⴳⵯ', 'ⵀ', 'ⵃ', 'ⵊ', 'ⴽ', 'ⴽⵯ',
                'ⵍ','ⵎ','ⵏ', 'ⵇ', 'ⵔ', 'ⵕ', 'ⵙ', 'ⵚ', 'ⵜ', 'ⵟ', 'ⵡ', 'ⵅ', 'ⵢ', 'ⵣ','ⵥ', 'ⴻ', 'ⵉ', 'ⵓ']

training_data = [] # cette liste qui contiendra les données d'entraînement traitées
testing_data = [] # cette liste qui contiendra les données de test traitées


In [ ]:
! kaggle datasets download -d benaddym/amazigh-handwritten-character-database-amhcd

Dataset URL: https://www.kaggle.com/datasets/benaddym/amazigh-handwritten-character-database-amhcd
License(s): other
 72% 49.0M/67.9M [00:00<00:00, 83.2MB/s]
100% 67.9M/67.9M [00:00<00:00, 104MB/s] 


In [ ]:
!unzip amazigh-handwritten-character-database-amhcd.zip -d ./amazigh-handwritten-character-database-amhcd

Streaming output truncated to the last 5000 lines.
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_515.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_516.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_517.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_518.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_519.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_52.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_520.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_521.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64/yax/yax_522.jpeg  
  inflating: ./amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Set the paths to the dataset directories
dataset_dir = "/content/amazigh-handwritten-character-database-amhcd/amhcd_64/AMHCD_64"
image_size = (64, 64)  # Resize all images to 64x64

# Function to preprocess images
def preprocess_image(image_path, img_size):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
    # Resize the image
    img_resized = cv2.resize(img, img_size)
    # Normalize the image
    img_normalized = img_resized / 255.0
    return img_normalized

# Create lists to store images and labels
images = []
labels = []

# Loop through each letter folder in the dataset directory
for letter_folder in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, letter_folder)

    if os.path.isdir(folder_path):  # Check if it's a directory
        # Loop through each image in the letter folder
        for img_file in tqdm(os.listdir(folder_path), desc=f"Processing {letter_folder}"):
            img_path = os.path.join(folder_path, img_file)
            # Preprocess the image
            img_preprocessed = preprocess_image(img_path, image_size)
            # Append the preprocessed image and label
            images.append(img_preprocessed)
            labels.append(letter_folder)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Reshape images to fit the model input (adding a channel dimension)
images = images.reshape(images.shape[0], image_size[0], image_size[1], 1)

# Encode the labels to numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert the labels to one-hot encoded format
labels_categorical = to_categorical(labels_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(labels)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Optionally, save the model
model.save("tifinagh_handwritten_recognition_model.h5")

# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Processing yadd: 100%|██████████| 780/780 [00:00<00:00, 3083.29it/s]
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 246s 382ms/step - accuracy: 0.3256 - loss: 2.4169 - val_accuracy: 0.9394 - val_loss: 0.2498
Epoch 2/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 258s 376ms/step - accuracy: 0.8424 - loss: 0.5004 - val_accuracy: 0.9578 - val_loss: 0.1433
Epoch 3/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 256s 366ms/step - accuracy: 0.8994 - loss: 0.3247 - val_accuracy: 0.9695 - val_loss: 0.1008
Epoch 4/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 255s 356ms/step - accuracy: 0.9217 - loss: 0.2472 - val_accuracy: 0.9736 - val_loss: 0.0943
Epoch 5/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 263s 358ms/step - accuracy: 0.9314 - loss: 0.2150 - val_accuracy: 0.9771 - val_loss: 0.0804
Epoch 6/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 267s 366ms/step - accuracy: 0.9414 - loss: 0.1856 - val_accuracy: 0.9775 - val_loss: 0.0774
Epoch 7/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 261s 364ms/step - accuracy: 0.9490 - loss: 0.1634 - val_accuracy: 0.9812 - val_loss: 0.0730
Epoch 8/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 230s 357ms/step - accuracy: 0.9525 -